In [9]:
from datasets import load_dataset, Dataset
import pandas as pd
import os
import json
import xmltodict
def clean_data(text):
    text = text.replace('{vocalsound}', '')
    text = text.replace('{disfmarker}', '')
    text = text.replace('a_m_i_', 'ami')
    text = text.replace('l_c_d_', 'lcd')
    text = text.replace('p_m_s', 'pms')
    text = text.replace('t_v_', 'tv')
    text = text.replace('{pause}', '')
    text = text.replace('{nonvocalsound}', '')
    text = text.replace('{gap}', '')
    return text

In [53]:
keyword_dict = {
    'dataset': [],
    'file_id': [],
    'text': [],
    'summary': []
}

for dataset in os.listdir('../data/KeywordExtractor-Datasets/datasets/'):
    path = os.path.join('../data/KeywordExtractor-Datasets/datasets/', dataset)
    for txt_file in os.listdir(os.path.join(path, 'docsutf8')):
        file_id = txt_file.replace('.txt', '')
        try:
            keyword_dict['dataset'].append(dataset)
            keyword_dict['file_id'].append(file_id)
            
            with open(os.path.join(path, 'docsutf8', txt_file)) as f:
                lines = f.readlines()
                striped_txt = []
                for line in lines:
                    striped_txt.append(line.strip())
                keyword_dict['text'].append(''.join(striped_txt))
            
            with open(os.path.join(path, 'keys', f'{file_id}.key')) as f:
                lines = f.readlines()
                striped_keys = []
                for line in lines:
                    striped_keys.append(line.strip())
                keyword_dict['summary'].append(','.join(striped_keys))
        except:
            print(f'could not process file: {txt_file}')


In [54]:
df = pd.DataFrame(keyword_dict)

In [72]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)
train.to_csv('/Users/silas.rudolf/projects/School/MA/experiments/data/keyword-extraction/train.csv', index=False)
test.to_csv('/Users/silas.rudolf/projects/School/MA/experiments/data/keyword-extraction/test.csv', index=False)

In [4]:
dataset = load_dataset("samsum")
dataset_kw = load_dataset("51la5/keyword-extraction")

Dataset samsum downloaded and prepared to /Users/silas.rudolf/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 205.04it/s]


In [87]:
df_kw = dataset_kw['train'].to_pandas().append(dataset_kw['test'].to_pandas())
df_kw['type'] = 'KEYWORD'

In [88]:
df_sum = dataset['train'].to_pandas().append(dataset['validation'].to_pandas()).append(dataset['test'].to_pandas())
df_sum['type'] = 'KEYPHRASE'

In [91]:
def qm_(path):
    keyword_dict = {
        'dataset': [],
        'file_id': [],
        'text': [],
        'summary': [],
        'type': []
    }
    for f in os.listdir(path):
        with open(os.path.join(path, f)) as t:
            transcript = json.load(t)

            for topic in transcript['topic_list']:
                keyword_dict['dataset'].append('QMSum')
                keyword_dict['file_id'].append(f)

                span_a, span_b = topic['relevant_text_span'][0]
                spans = transcript['meeting_transcripts'][int(span_a): int(span_b)]
                text = []
                for utterance in spans:
                    text.append(f"{utterance['speaker']}: {clean_data(utterance['content'])}")
                keyword_dict['text'].append(''.join(text))
                keyword_dict['summary'].append(topic['topic'])
                keyword_dict['type'].append('KEYPHRASE')

    return pd.DataFrame(keyword_dict)

In [93]:
path_train = '/Users/silas.rudolf/projects/School/MA/experiments/data/QMSum/data/ALL/train'
path_val = '/Users/silas.rudolf/projects/School/MA/experiments/data/QMSum/data/ALL/val'
path_test = '/Users/silas.rudolf/projects/School/MA/experiments/data/QMSum/data/ALL/test'

qm_t = qm_(path_train)
qm_val = qm_(path_val)
qm_test = qm_(path_test)

In [106]:
qm_df = qm_t.append(qm_val).append(qm_test)
df_sum.rename(columns={'id': 'file_id', 'dialogue': 'text'}, inplace=True)

In [108]:
df = df_kw.append(df_sum).append(qm_df)

In [317]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)
train = train.drop(train[train.text == ''].index).dropna(how='all')
test = test.drop(test[test.text == ''].index).dropna(how='all')

train.to_csv('/Users/silas.rudolf/projects/School/MA/experiments/data/keyword-extraction/train.csv', index=False)
test.to_csv('/Users/silas.rudolf/projects/School/MA/experiments/data/keyword-extraction/test.csv', index=False)

In [189]:
raw_datasets = load_dataset("51la5/keyword-extraction")


Using custom data configuration 51la5--keyword-extraction-e9f2eb93d6495740
Reusing dataset csv (/Users/silas.rudolf/.cache/huggingface/datasets/51la5___csv/51la5--keyword-extraction-e9f2eb93d6495740/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)
100%|██████████| 2/2 [00:00<00:00, 74.45it/s]


In [248]:
with open("/Users/silas.rudolf/projects/School/MA/experiments/data/ake-datasets/datasets/ACM/test/1005058.xml") as xml_file:
    data_dict = xmltodict.parse(xml_file.read())

In [303]:
def ake_(path, prefix):
    keyword_dict = {
        'dataset': [],
        'file_id': [],
        'text': [],
        'summary': [],
        'type': []
    }
    with open(os.path.join(path, f'references/{prefix}.combined.json')) as f:
        references = json.load(f)

    for filename in os.listdir(os.path.join(path, f'{prefix}')):
        file_id = filename.replace('.xml', '')
        with open(os.path.join(path, f'{prefix}', filename)) as xml_file:
            doc = xmltodict.parse(xml_file.read())

        sentences = []
        for sentence in doc['root']['document']['sentences']['sentence']:
            try:
                sentences.append(' '.join([i['word'] for i in sentence['tokens']['token']]))
            except:
                pass
        keyword_dict['dataset'].append(path.split('/')[-1])
        keyword_dict['file_id'].append(file_id)
        keyword_dict['text'].append(''.join(sentences))
        keyword_dict['summary'].append(','.join([i[0] for i in references[file_id]]))
        keyword_dict['type'].append('KEYWORD')
    return keyword_dict

In [304]:
t = ake_('/Users/silas.rudolf/projects/School/MA/experiments/data/ake-datasets/datasets/NUS', 'test')


In [305]:
pd.DataFrame(t).to_csv('/Users/silas.rudolf/projects/School/MA/experiments/data/ake-datasets/df5.csv', index=False)

In [307]:
df1 = pd.read_csv('/Users/silas.rudolf/projects/School/MA/experiments/data/ake-datasets/df1.csv')
df2 = pd.read_csv('/Users/silas.rudolf/projects/School/MA/experiments/data/ake-datasets/df2.csv')
df3 = pd.read_csv('/Users/silas.rudolf/projects/School/MA/experiments/data/ake-datasets/df3.csv')
df4 = pd.read_csv('/Users/silas.rudolf/projects/School/MA/experiments/data/ake-datasets/df4.csv')
df5 = pd.read_csv('/Users/silas.rudolf/projects/School/MA/experiments/data/ake-datasets/df5.csv')

In [311]:
df_new = df1.append(df2).append(df3).append(df4).append(df5)

In [351]:
raw_datasets = load_dataset("51la5/keyword-extraction")

Using custom data configuration 51la5--keyword-extraction-e9f2eb93d6495740
Reusing dataset csv (/Users/silas.rudolf/.cache/huggingface/datasets/51la5___csv/51la5--keyword-extraction-e9f2eb93d6495740/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)
100%|██████████| 2/2 [00:00<00:00, 175.49it/s]


In [316]:
df = raw_datasets['train'].to_pandas().append(raw_datasets['test'].to_pandas()).append(df_new)

In [339]:
x = raw_datasets.filter(lambda x: x['type'] == 'KEYWORD')
y = raw_datasets.filter(lambda x: x['type'] == 'KEYPHRASE')


100%|██████████| 7/7 [00:00<00:00, 50.26ba/s]


In [340]:
kwords = [i for i in x['train']['text']]
kphrase = [i for i in y['train']['text']]

In [350]:
sum(x['test'].to_pandas()['dataset'] == None)

0

In [353]:
raw_datasets['test'].to_pandas()

,dataset,file_id,text,summary,type
0,None,13680597,"Jimmy: Sorry, have you seen Maria today?\r\nPe...",Peter saw Maria briefly today and he thinks sh...,KEYPHRASE
1,Schutz2008,Arch_Orthop_Trauma_Surg-4-1-2225998,Arch Orthop Trauma SurgArchives of Orthopaedic...,"Classification,Radius,Radius fracture,X-ray,Di...",KEYWORD
2,None,13864482,"Eva: Michelle has a huge stain on her bum, has...",Michelle has a stain on her bum. Harriett will...,KEYPHRASE
3,None,13727696,"Daniel: beer?\r\nDavid: oh, man, sure\r\nDanie...",Daniel will bring some beers for himself and D...,KEYPHRASE
4,None,13716006,"Faith: Everybody, look, Lottie's back! :D\r\nL...",Lottie went on a date with Tommy but is not su...,KEYPHRASE
...,...,...,...,...,...
5508,None,13682232,Anna: <file_video>\r\nKate: :-) New song?\r\nA...,Anna is crazy about the new song.,KEYPHRASE
5509,None,13731478,Mary: don't forget to pick up your sister!\r\n...,Freddie will pick up his sister. Mary will buy...,KEYPHRASE
5510,SemEval2010,J-55,From Optimal Limited To Unlimited Supply Aucti...,"mechanism design,auction,competitive analysis,...",KEYWORD
5511,None,13729357,"Dot: Hi Jimmy, I have a favour to ask\r\nJimmy...",Ted was supposed to take care of Dot's cats un...,KEYPHRASE


In [22]:
import os
import json
path = '/Users/silas.rudolf/projects/School/MA/experiments/data/QMSum/data/ALL/test'
files = os.listdir(path)

keyphrases_dict = {
    'filename': [],
    'text': [],
    'summary': []
}

for filename in files:
    with open(os.path.join(path, filename)) as f:
        json_file = json.load(f)
        keyphrases_dict['filename'].append(filename)

        text = []
        for utterance in json_file['meeting_transcripts']:
            text.append(f"{utterance['speaker']}: {clean_data(utterance['content'])}")
        keyphrases_dict['text'].append(''.join(text))
        keyphrases_dict['summary'].append(json_file['general_query_list'][0]['answer'])


path = '/Users/silas.rudolf/projects/School/MA/experiments/data/QMSum/data/ALL/val'
files = os.listdir(path)
for filename in files:
    with open(os.path.join(path, filename)) as f:
        json_file = json.load(f)
        keyphrases_dict['filename'].append(filename)

        text = []
        for utterance in json_file['meeting_transcripts']:
            text.append(f"{utterance['speaker']}: {clean_data(utterance['content'])}")
        keyphrases_dict['text'].append(''.join(text))
        keyphrases_dict['summary'].append(json_file['general_query_list'][0]['answer'])

path = '/Users/silas.rudolf/projects/School/MA/experiments/data/QMSum/data/ALL/train'
files = os.listdir(path)
for filename in files:
    with open(os.path.join(path, filename)) as f:
        json_file = json.load(f)
        keyphrases_dict['filename'].append(filename)

        text = []
        for utterance in json_file['meeting_transcripts']:
            text.append(f"{utterance['speaker']}: {clean_data(utterance['content'])}")
        keyphrases_dict['text'].append(''.join(text))
        keyphrases_dict['summary'].append(json_file['general_query_list'][0]['answer'])



In [24]:
df = pd.DataFrame(keyphrases_dict)
df.to_csv('/Users/silas.rudolf/projects/School/MA/experiments/data/keyphrase-eval/reference.csv', index=False)